In [9]:
import cv2
import numpy as np
import os

# === Load and preprocess image ===
image_path = 'test images/malignant3.jpg'
img = cv2.imread(image_path)
img = cv2.resize(img, (700, 700))
original_image = img.copy()  # for saving unmodified original
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# === Thresholding using Otsu's method ===
ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# === Noise removal with morphological opening ===
kernel = np.ones((3, 3), np.uint8)
opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)

# === Sure background by dilation ===
sure_bg = cv2.dilate(opening, kernel, iterations=3)

# === Sure foreground by distance transform ===
dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
ret, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)

# === Unknown region ===
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg, sure_fg)

# === Marker labelling ===
ret, markers = cv2.connectedComponents(sure_fg)
markers = markers + 1  # background=1, objects start from 2
markers[unknown == 255] = 0  # unknown region set to 0

# === Apply Watershed ===
img_watershed = img.copy()
markers = cv2.watershed(img_watershed, markers)

# === Create binary mask (segmented objects only) ===
binary_mask = np.uint8(markers > 1) * 255

# === Create segmented image using mask ===
segmented = cv2.bitwise_and(original_image, original_image, mask=binary_mask)

# === Draw watershed boundaries on a copy ===
img_with_boundaries = original_image.copy()
img_with_boundaries[markers == -1] = [255, 255, 255]  # white watershed lines

#Save images
output_dir = 'segmentation_output'
os.makedirs(output_dir, exist_ok=True)
cv2.imwrite(os.path.join(output_dir, 'original_image.png'), original_image)
cv2.imwrite(os.path.join(output_dir, 'binary_mask.png'), binary_mask)
cv2.imwrite(os.path.join(output_dir, 'segmented_image.png'), segmented)
cv2.imwrite(os.path.join(output_dir, 'watershed_boundaries.png'), img_with_boundaries)

#Resize for display (e.g., 350x350)
def resize_for_display(image, size=(350, 350)):
    return cv2.resize(image, size)

# === Add labels ===
label_font = cv2.FONT_HERSHEY_SIMPLEX
def label(img, text):
    return cv2.putText(img.copy(), text, (10, 30), label_font, 0.8, (0, 255, 0), 2)

# === Resize and label each output ===
original_disp = label(resize_for_display(original_image), "Original")
mask_disp = label(cv2.cvtColor(resize_for_display(binary_mask), cv2.COLOR_GRAY2BGR), "Binary Mask")
seg_disp = label(resize_for_display(segmented), "Segmented")
boundary_disp = label(resize_for_display(img_with_boundaries), "Boundaries")

# === Horizontally stack them ===
horizontal_display = np.hstack([original_disp, mask_disp, seg_disp, boundary_disp])

# === Display in one horizontal row ===
cv2.imshow('Watershed Segmentation Results - Techstack', horizontal_display)
cv2.waitKey(0)
cv2.destroyAllWindows()
